In [9]:
%load_ext autoreload
%autoreload 2

from time import time
import pandas as pd
import numpy as np
import os
from collections import Counter, defaultdict
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pyterrier as pt
pt.init()

PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
from jnius import autoclass
tokeniser = autoclass("org.terrier.indexing.tokenisation.Tokeniser").getTokeniser()

In [4]:
def terrier_tokenizer(raw_utterance):
    new_utterance = " ".join(tokeniser.getTokens(raw_utterance))
    return new_utterance

# Retrieve docs per MANUAL query

In [5]:
%%time
index_ref = pt.IndexRef.of("/data3/muntean/TREC_CAST_2022/indexes/CAST2022-stemmed/data.properties")
index = pt.IndexFactory.of(index_ref)

print(index.getCollectionStatistics())

14:21:30.770 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
Number of documents: 106400940
Number of terms: 22580112
Number of postings: 7942682174
Number of fields: 1
Number of tokens: 12168635367
Field names: [text]
Positions:   false

CPU times: user 21.9 s, sys: 28.1 s, total: 50 s
Wall time: 44.7 s


In [ ]:
# qrel_path = "../data/CAST_qrels/qrels-docs.2019.txt"
# qrels_df = pd.read_csv(qrel_path, delimiter=" ", header=None)
# qrels_df[[3]] = qrels_df[[3]].astype(int)
# qrels_df = qrels_df.drop([1], axis=1)
# qrels_df.columns=["qid", "docno", "label"]
# qrels = qrels_df

In [6]:
def get_topics(topics_path):
    # topics_path='./data/queries.CAST2022manual.test.tsv' #manual

    topics_df = pd.read_csv(topics_path, delimiter="\t", header=None)

    topics_df[2] = topics_df[1].apply(lambda s: terrier_tokenizer(s))

    topics_df = topics_df.drop([1], axis=1)
    topics_df.columns=["qid", "query"]

    topics = topics_df
    # print(topics.head())
    return topics

In [7]:
def trec_format_to_dict_for_TRECjson(run_name, run_type, res):
    result_dict= {}
    # level 0
    result_dict["run_name"]=run_name
    result_dict["run_type"]=run_type
    result_dict["turns"]= []
    
    qids = list(res['qid'].unique())
    
    for turn_id in qids:
        turn_dict = {"turn_id": turn_id}
        turn_res = res[res["qid"]==turn_id].reset_index()
        responses = []
        response = {}
        resp = turn_res[turn_res["rank"]==0]["text"].values[0] + " " + turn_res[turn_res["rank"]==1]["text"].values[0] + " " + turn_res[turn_res["rank"]==2]["text"].values[0]
        response["text"] =  " ".join(resp.replace("\"", "").split(" ")[:250])
        response["rank"] = 1
        response["provenance"] = []
        for i in range(0,len(turn_res)):
            ranked_doc = {"id" :turn_res["docno"][i], "text":turn_res["text"][i] , "score":turn_res["score"][i] }
            response["provenance"].append(ranked_doc)
        responses.append(response)
        turn_dict["responses"] = responses
        result_dict["turns"].append(turn_dict)
    
    return result_dict

In [12]:
%%time
# DEPRECATED, writes TREC format
path = "./data/"
runs = [
        #("queries.CAST2022.run1.prevraw.test.tsv", "hpc_cnr_run1", "automatic"), 
         #("queries.CAST2022.run2.prevenriched.test.tsv", "hpc_cnr_run2", "automatic"), 
         #("queries.CAST2022.run3.prevauto.test.tsv", "CNR_run1", "automatic"), 
         # ("queries.CAST2022.run4.prevmanual.test.tsv", "CNR_run2", "manual"),
         ("queries.CAST2022.run5.rawithresponse1sent.test.tsv", "CNR_run3", "automatic"),
         ("queries.CAST2022.run6.rawithresponsetop5tokens.test.tsv", "CNR_run4", "automatic"),
        ]

for run in runs:
    file = run[0]
    topics = get_topics(path + file)
    DPH = pt.BatchRetrieve(index, wmodel="DPH", num_results=1000, metadata=["docno","text"])  
    res = DPH.transform(topics)
    pt.io.write_results(res,"./runs/"+file+".trec")
    
    result_dict = trec_format_to_dict_for_TRECjson(run[1], run[2], res)
    with open("./runs/"+file+".json", "w") as write_file:
        json.dump(result_dict, write_file, indent=4)

CPU times: user 25min 29s, sys: 1min 7s, total: 26min 37s
Wall time: 39min 45s


In [ ]:
# %%time
# pt.Experiment([DPH], topics, qrels, names=["DPH"], 
#               eval_metrics=["map", "recip_rank", "recall_200", "P_3", "P_1", "ndcg_cut_3"])

In [ ]:
# %%time
# res_per_query = pt.Experiment([DPH], topics, qrels, names=["DPH"], 
#               eval_metrics=["map", "recip_rank", "recall_200", "P_3", "P_1", "ndcg_cut_3"], perquery=True)

In [ ]:
# res_per_query.head(50)

# RES to JSON

In [ ]:
import pandas as pd

df = pd.read_csv("data/res_example.tsv", delimiter="\t")
#df = res

qids = df['qid'].unique()
for qid in qids:
    print(qid)

In [ ]:
df

In [ ]:
j = (df.groupby(['qid'])
    .apply(lambda x: x[['summary','rank_summary']].to_dict('records'))
    .reset_index()
    .rename(columns={0:'responses'})
    .to_json(orient='records')
    )
j = j.replace("\"summary\":", "\"text\":").replace("\"rank_summary\":", "\"rank\":")

j2 = (df.groupby(['qid','rank_summary','summary'])
    .apply(lambda x: x[['docno','text','score']].to_dict('records'))
    .reset_index()
    .rename(columns={0:'provenance'})
    .to_json(orient='records')
    )
j2 = j2.replace("\"summary\":", "\"text\":").replace("\"rank_summary\":", "\"rank\":").replace("\"qid\":", "\"turn_id\":").replace("\"docno\":", "\"id\":")


In [ ]:
print(j)

import json
with open("data/j.json", "w") as write_file:
    json.dump(j, write_file, indent=4)

In [ ]:
print(j2)

import json
with open("data/j2.json", "w") as write_file:
    json.dump(j2, write_file, indent=4)

In [ ]:
#nested example from
#https://stackoverflow.com/questions/61781186/pandas-grouping-by-multiple-columns-to-get-a-multi-nested-json

df_example = pd.read_csv("data/dummy_example.tsv", delimiter="\t")

print(df_example)

test = [df_example.groupby(['lvl1'])
        .apply(lambda x: x.groupby(['lvl2'])
        .apply(lambda x: [x.groupby(['lvl3'])
                          .apply(lambda x: x[['lvl4','lvl5']].to_dict('r')
                        ).to_dict()]
                         ).to_dict()
          ).to_dict()]


In [ ]:
test

In [ ]:
#for writing in json file
import json

with open("data/dummy_example.json", "w") as write_file:
    json.dump(test, write_file, indent=4)